In [2]:
import pandas as pd
from thefuzz import process

In [4]:
START_YEAR = 2021
END_YEAR = 2050
DEVOLVED_AUTHS = ['United Kingdom', 'Scotland', 'Wales', 'Northern Ireland']
GASES = ['CARBON', 'CH4', 'N2O']
SD_COLUMNS = ['Measure ID', 'Country', 'Sector', 'Subsector', 'Category1 EE_sector', 'Category2 Process', 'Category3 Tech', 'Measure Name', 'Measure Variable', 'Variable Unit']
SD_COLUMNS += list(range(START_YEAR, END_YEAR+1))
OUTPUT_FILE = 'sd-industry-test.xlsx'

### Load data

#### Step 1: Get measure definitions

In [21]:
def get_subsectors(fpath, ccc_sector):
    # read the excel file
    with open(fpath, 'rb') as f:
        nzip_defs_df = pd.read_excel(f, 'Sheet1')

    # check the ccc sector is valid
    if ccc_sector not in nzip_defs_df['CCC sector'].unique():
        raise ValueError(f'Invalid CCC sector: {ccc_sector}, must be one of {nzip_defs_df["CCC sector"].unique()}')

    # filter rows based on the CCC sector we are interested in
    sector_defs = nzip_defs_df.loc[nzip_defs_df['CCC sector'] == ccc_sector]

    # return the EE sectors which correspond to this CCC sector
    return sector_defs['EE sector'].unique().tolist()

#### Step 2: load NZIP Outputs

In [34]:
def load_nzip(nzip_path: str, sector_defs_path: str, sector: str):
    """
    Load NZIP outputs and filter rows based on CCC sector.
    Some initial data cleaning is also applied.

    Parameters
    ----------
    nzip_path : str
        The path to the NZIP output workbook.
    sector_defs_path : str
        The path to the NZIP sector definitions workbook.
    sector : str
        CCC sector to load. Must be one of 'Industry', 'Fuel Supply', or 'Waste'.
    """

    # get the list of subsectors for this CCC sector. these will be used to filter the NZIP outputs
    subsectors = get_subsectors(sector_defs_path, sector)

    # read the NZIP output workbook. this can take a minute or two, we can speed it up by first converting it to a csv
    with open(nzip_path, 'rb') as f:
        df = pd.read_excel(f, sheet_name='CCC Outputs', header=10, usecols='F:CWV')

    # select relevant rows based on the sector
    df = df.loc[df['Element_sector'].isin(subsectors)]

    # fix string columns which have some empty cells
    df['Selected Option'] = df['Selected Option'].fillna('')
    df['Technology Type'] = df['Technology Type'].fillna('')

    # fix some numeric columns which have some non-numeric values (these values will be set to 0 later)
    fix_numeric_cols = ['% CARBON Emissions', '% CH4 Emissions', '% N2O Emissions']
    fix_numeric_cols += [f'Total AM costs (£m) {y}' for y in range(START_YEAR, END_YEAR+1)]
    fix_numeric_cols += [f'AM opex (£m) {y}' for y in range(START_YEAR, END_YEAR+1)]
    fix_numeric_cols += [f'AM fuel costs (£m) {y}' for y in range(START_YEAR, END_YEAR+1)]
    for col in fix_numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # any NaN (not a number) values are set to 0
    df = df.fillna(0)
    
    return df

In [35]:
nzip_path = 'N-ZIP-Model_version1_2_AG_updated_19_12_2023.xlsb'
sector_defs_path = 'NZIP_Sector_Process.xlsx'
sector = 'Industry'
df = load_nzip(nzip_path, sector_defs_path, sector)

In [36]:
def col_search(df, search_string, limit=5):
    """
    Search the columns of a dataframe for a string using thefuzz.

    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe to search.
    search_string : str
        The string to search for.
    limit : int
        The maximum number of results to return.
    """
    return process.extract(search_string, df.columns, limit=limit)

### Abatement emissions

In [37]:
sector_map = {
 'Cement': 'Cement and Lime',
 'Compressor Station': 'Fossil fuel production',
 'Ethylene': 'Chemicals',
 'Food & Drink': 'Food, Drink, Tobacco',
 'Gas Terminal': 'Fossil fuel production',
 'Glass': 'Glass and other minerals',
 'Iron (Port Talbot Scunthorpe)': 'Iron & Steel',
 'Lime': 'Cement and Lime',
 'LNG Terminal': 'Fossil fuel production',
 'Non ferrous metal': 'Non ferrous metals',
 'Gas Platform': 'Fossil fuel production',
 'Oil Terminal': 'Fossil fuel production',
 'Other Chemicals': 'Chemicals',
 'Other Fuel Production': 'Fossil fuel production',
 'Other industry': 'Other manufacturing and construction',
 'Other Iron and Steel': 'Iron & Steel',
 'Other Minerals': 'Glass and other minerals',
 'Paper': 'Paper, Pulp, Print',
 'Refining': 'Refining',
 'Vehicles': 'Vehicles',
 'Waste Processing': 'Water and waste management',
 'Coal Mine (closed)': 'Coal mines',
 'Coal Mine (open)': 'Coal mines',
 'Gas Distribution': 'Fossil fuel production',
 'Construction': 'Other manufacturing and construction',
 'Ammonia': 'Chemicals',
 'Oil Platform': 'Fossil fuel production',
 'Shale Gas': 'Fossil fuel production',
 'Off-road mobile machinery': 'Off-road mobile machinery'
}


In [40]:
def sector_databook_format(df, variable_name, variable_unit):
    df = df.reset_index()
    df['Measure ID'] = ''
    df['Sector'] = 'Industry'
    df['Subsector'] = df['Element_sector'].map(sector_map)
    df['Measure Name'] = df['Element_sector'] + '_' + df['Process'] + '_' + df['Selected Option']
    df['Measure Variable'] = variable_name
    df['Variable Unit'] = variable_unit
    df['Category1 EE_sector'] = df['Element_sector']
    df['Category2 Process'] = df['Process']
    df['Category3 Tech'] = df['Selected Option']
    df = df[SD_COLUMNS]
    return df

def aggregate_timeseries_country(df, timeseries, variable_name, variable_unit, weight_col=None, country='United Kingdom', scale=None):

    # get the emissions time series columns
    total_emissions_cols = [f'{timeseries} {y}' for y in range(START_YEAR, END_YEAR+1)]
    emissions_cols = list(range(START_YEAR, END_YEAR+1))
    df[emissions_cols] = df[total_emissions_cols].copy()

    # multiply by another column and/or then scale by a fixed value
    if weight_col:
        df[emissions_cols] = df[emissions_cols].multiply(df[weight_col], axis=0)
    if scale:
        df[emissions_cols] = df[emissions_cols] * scale

    # sum rows corresponding to the same measure
    agg_emissions_df = df.groupby(['Element_sector', 'Process', 'Selected Option'])[emissions_cols].sum()

    # add country column
    agg_emissions_df['Country'] = country

    # format as sector databook
    df = sector_databook_format(agg_emissions_df, variable_name, variable_unit)

    return df

def aggregate_timeseries(df, **kwargs):
    # go through each country and combine the results
    dfs = [aggregate_timeseries_country(df, country=country, **kwargs) for country in DEVOLVED_AUTHS]
    df = pd.concat(dfs)
    return df

In [41]:
# add capex and opex cols
for y in range(START_YEAR, END_YEAR+1):
    df[f'capex {y}'] = df[f'AM capex (£m) {y}'] - df[f'Counterfactual capex (£m) {y}']
    df[f'opex {y}'] = df[f'AM opex (£m) {y}'] + df[f'AM fuel costs (£m) {y}'] - (df[f'Counterfactual opex (£m) {y}'] + df[f'Counterfactual fuel costs (£m) {y}'])

In [42]:
function_calls = [
    # Add total direct and indirect emissions
    {
        "timeseries": "Total direct emissions abated (MtCO2e)",
        "variable_name": "Abatement total direct",
        "variable_unit": "MtCO2e",
    },
    {
        "timeseries": "Total indirect emissions abated (MtCO2e)",
        "variable_name": "Abatement total indirect",
        "variable_unit": "MtCO2e",
    },

    # Add emissions by gas
    {
        "timeseries": "Total direct emissions abated (MtCO2e)",
        "variable_name": "Abatement emissions CO2",
        "variable_unit": "MtCO2",
        "weight_col": "% CARBON Emissions",
    },
    {
        "timeseries": "Total direct emissions abated (MtCO2e)",
        "variable_name": "Abatement emissions CH4",
        "variable_unit": "MtCO2e",
        "weight_col": "% CH4 Emissions",
    },
    {
        "timeseries": "Total direct emissions abated (MtCO2e)",
        "variable_name": "Abatement emissions N20",
        "variable_unit": "MtCO2e",
        "weight_col": "% N2O Emissions",
    },

    # Add demand
    {
        "timeseries": "Change in electricity use (GWh)",
        "variable_name": "Additional demand electricity",
        "variable_unit": "TWh",
        "scale": 1e-3,
    },
    {
        "timeseries": "Change in natural gas use (GWh)",
        "variable_name": "Additional demand gas",
        "variable_unit": "TWh",
        "scale": 1e-3,
    },
    {
        "timeseries": "Change in petroleum use (GWh)",
        "variable_name": "Additional demand petroleum",
        "variable_unit": "TWh",
        "scale": 1e-3,
    },
    {
        "timeseries": "Change in solid fuel use (GWh)",
        "variable_name": "Additional demand solid fuel",
        "variable_unit": "TWh",
        "scale": 1e-3,
    },
    {
        "timeseries": "Change in primary bioenergy use (GWh)",
        "variable_name": "Additional demand final bioenergy",
        "variable_unit": "TWh",
        "scale": 1e-3,
    },
    {
        "timeseries": "Change in hydrogen use (GWh)",
        "variable_name": "Additional demand hydrogen",
        "variable_unit": "TWh",
        "scale": 1e-3,
    },

    # Add capex and opex
    {
        "timeseries": "capex",
        "variable_name": "Additional capital expenditure",
        "variable_unit": "£m",
    },
    {
        "timeseries": "opex",
        "variable_name": "Additional operational expenditure",
        "variable_unit": "£m",
    },
]

In [57]:
sd_df = pd.DataFrame(columns=SD_COLUMNS)
for args in function_calls:
    sd_df = pd.concat([sd_df, aggregate_timeseries(df, **args)])

In [58]:
sd_df.head()

,Measure ID,Country,Sector,Subsector,Category1 EE_sector,Category2 Process,Category3 Tech,Measure Name,Measure Variable,Variable Unit,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,,United Kingdom,Industry,Chemicals,Ammonia,Combustion CO2 - Reforming,,Ammonia_Combustion CO2 - Reforming_,Abatement total direct,MtCO2e,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,,United Kingdom,Industry,Chemicals,Ammonia,Combustion CO2 - Reforming,CCS - Advanced amines or blends,Ammonia_Combustion CO2 - Reforming_CCS - Advan...,Abatement total direct,MtCO2e,...,0.245961,0.245905,0.245849,0.245793,0.245738,0.245690,0.245643,0.245595,0.245548,0.245500
2,,United Kingdom,Industry,Chemicals,Ammonia,Combustion CO2 - Reforming,CCS - Calcium Looping,Ammonia_Combustion CO2 - Reforming_CCS - Calci...,Abatement total direct,MtCO2e,...,0.164822,0.164784,0.164747,0.164709,0.164672,0.164640,0.164608,0.164576,0.164545,0.164513
3,,United Kingdom,Industry,Chemicals,Ammonia,Process CO2 - Reforming,,Ammonia_Process CO2 - Reforming_,Abatement total direct,MtCO2e,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,,United Kingdom,Industry,Chemicals,Ammonia,Process CO2 - Reforming,CCS - Advanced amines or blends,Ammonia_Process CO2 - Reforming_CCS - Advanced...,Abatement total direct,MtCO2e,...,0.694148,0.694148,0.694148,0.694148,0.694148,0.694148,0.694148,0.694148,0.694148,0.694148


In [62]:
sd_df.to_excel(OUTPUT_FILE, index=False, sheet_name='BP Measure level data')

In [47]:
# write a sheet containing the measure definitions
measure_defs_df = pd.DataFrame({
    'Sector': pd.Series(sd_df['Sector'].unique()).sort_values(),
    'Subsector': pd.Series(sd_df['Subsector'].unique()).sort_values(),
    'EE_sector': pd.Series(sd_df['Category1 EE_sector'].unique()).sort_values(),
    'Process': pd.Series(sd_df['Category2 Process'].unique()).sort_values(),
    'Tech': pd.Series(sd_df['Category3 Tech'].unique()).sort_values(),
    'Measure Name': pd.Series(sd_df['Measure Name'].unique()).sort_values(),
})

with pd.ExcelWriter(OUTPUT_FILE, mode='a', if_sheet_exists='replace') as writer:
    measure_defs_df.to_excel(writer, index=False, sheet_name='Measure definitions')


### Baseline emissions

In [48]:
bl_cols = [
    {
        "timeseries": "Baseline emissions (MtCO2e)",
        "variable_name": "Baseline emissions CO2",
        "variable_unit": "MtCO2",
        "weight_col": "% CARBON Emissions",
    },
    {
        "timeseries": "Baseline emissions (MtCO2e)",
        "variable_name": "Baseline emissions CH4",
        "variable_unit": "MtCO2e",
        "weight_col": "% CH4 Emissions",
    },
    {
        "timeseries": "Baseline emissions (MtCO2e)",
        "variable_name": "Baseline emissions N20",
        "variable_unit": "MtCO2e",
        "weight_col": "% N2O Emissions",
    },
]

In [49]:
bl_df = pd.DataFrame(columns=SD_COLUMNS)
for args in bl_cols:
    bl_df = pd.concat([bl_df, aggregate_timeseries(df, **args)])


In [50]:
bl_df = bl_df.groupby(['Country', 'Sector',  'Subsector', 'Measure Variable', 'Variable Unit']).sum(numeric_only=True)
bl_df = bl_df.reset_index()
bl_df = bl_df.rename(columns={'Measure Variable': 'Baseline Variable'})
with pd.ExcelWriter(OUTPUT_FILE, mode='a', if_sheet_exists='replace') as writer:
    bl_df.to_excel(writer, index=False, sheet_name='Baseline data')

### Aggregated outputs

In [51]:
agg_df = pd.DataFrame(columns=['Senario', 'Country', 'Sector', 'Aggregate Variable', 'Variable Unit'] + list(range(START_YEAR, END_YEAR+1)))

In [52]:
total_abatement = sd_df.loc[(sd_df['Measure Variable'] == 'Abatement total direct') & (sd_df['Country'] == 'United Kingdom')].sum(numeric_only=True)
total_baseline_emissions = bl_df.loc[(bl_df['Baseline Variable'] == 'Baseline emissions CO2') & (bl_df['Country'] == 'United Kingdom')].sum(numeric_only=True)
total_pathway_emissions = total_baseline_emissions - total_abatement

In [53]:
agg_df.loc['Baseline emissions total'] = total_baseline_emissions
agg_df.loc['Baseline emissions total', 'Aggregate Variable'] = 'Baseline emissions total'
agg_df.loc['Baseline emissions total', 'Variable Unit'] = 'MtCO2e'

agg_df.loc['Direct emissions total'] = total_pathway_emissions
agg_df.loc['Direct emissions total', 'Aggregate Variable'] = 'Direct emissions total'
agg_df.loc['Direct emissions total', 'Variable Unit'] = 'MtCO2e'


In [54]:
agg_df['Country'] = 'United Kingdom'
agg_df['Sector'] = 'Industry'

In [55]:
with pd.ExcelWriter(OUTPUT_FILE, mode='a', if_sheet_exists='replace') as writer:
    agg_df.to_excel(writer, index=False, sheet_name='Aggregate data')

## looking at measure definitions

In [25]:
industry_measures = set([industry_sectors[i] + '_' + industry_process[i] + '_' + industry_tech[i] for i in range(len(industry_sectors))])
nzip_measures = set(df['Element_sector'].astype(str) + '_' + df['Process'].astype(str) + '_' + df['Selected Option'].astype(str))
lost = set(industry_measures) - set(nzip_measures)

In [26]:
len(industry_measures), len(nzip_measures), len(lost)

(452, 145, 393)

In [27]:
# save list of lost measures to a text file in the same directory as the excel files
with open('lost_measures.txt', 'w') as f:
    # write using join
    f.write('\n'.join(sorted(lost)))